In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf

labels_dict = {
0:'云',
1:'京',
2:'冀',
3:'吉',
4:'宁',
5:'川',
6:'广',
7:'新',
8:'晋',
9:'桂',
10:'沪',
11:'津',
12:'浙',
13:'渝',
14:'湘',
15:'琼',
16:'甘',
17:'皖',
18:'粤',
19:'苏',
20:'蒙',
21:'藏',
22:'豫',
23:'贵',
24:'赣',
25:'辽',
26:'鄂',
27:'闽',
28:'陕',
29:'青',
30:'鲁',
31:'黑'}

#初始化权重
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
#初始化偏差
def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#卷积
def convolution(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#最大池化
def maxPool2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#进行卷积
#输入的数据和标签的占位符
x = tf.placeholder(tf.float32, [None, 784])
y_labels = tf.placeholder(tf.float32, shape=[None, 32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

##一层卷积
Wconvo1 = weight_var([5, 5, 1, 32])
bconvo1 = bias_var([32])

hconvo1 = tf.nn.relu(convolution(x_image, Wconvo1) + bconvo1)
hpool1 = maxPool2x2(hconvo1)
##二层卷积
Wconvo2 = weight_var([5, 5, 32, 64])
bconvo2 = bias_var([64])

hconvo2 = tf.nn.relu(convolution(hpool1, Wconvo2) + bconvo2)
hpool2 = maxPool2x2(hconvo2)

#密集连接层
Wfc1 = weight_var([7 * 7 * 64, 1024])
bfc1 = bias_var([1024])

hpool2flat = tf.reshape(hpool2, [-1, 7*7*64])
hfc1 = tf.nn.relu(tf.matmul(hpool2flat, Wfc1) + bfc1)

#防止过拟合操作,dropout(随机选取权重进行运算)
keep_prob = tf.placeholder("float")
hfc1drop = tf.nn.dropout(hfc1, keep_prob)

#output layer
Wfc2 = weight_var([1024, 32])
bfc2 = bias_var([32])

y_conv = tf.nn.softmax(tf.matmul(hfc1drop, Wfc2) + bfc2)

#交叉熵
cross_entropy = -tf.reduce_sum(y_labels*tf.log(y_conv))
#检查预测值是否正确
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_labels, 1))

#准确度计算
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#梯度下降
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


#将文件读取为一个队列
filename_queue = tf.train.string_input_producer(['./zh_character_test.tfrecords'])
reader = tf.TFRecordReader()

_, example = reader.read(filename_queue)
features = tf.parse_single_example(
    example,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image_raw':tf.FixedLenFeature([], tf.string)
    }
)

#还原图像和标签 
image = tf.decode_raw(features['image_raw'], tf.uint8) 
image = tf.reshape(image, [28, 28, 1])
label = tf.decode_raw(features['label'], tf.int64) 
label = tf.reshape(label, [32])

#打乱图像顺序
image_batch, label_batch = tf.train.batch([image, label],  
                                                batch_size=1,  
                                                capacity=20000,  
                                                num_threads=2)  
image = tf.reshape(image_batch, (1, 28*28))
label = tf.reshape(label_batch, (1, 32))

#保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #读取并使用模型
    ckpt = tf.train.get_checkpoint_state('./model')
    saver.restore(sess, ckpt.model_checkpoint_path)   

    la_dic = {i:[0,0] for i in range(32)}   
    for i in range(1114):
        img, la = sess.run([image, label])
        img = img / 255
        a = tf.argmax(y_labels, 1)
        b = tf.argmax(y_conv, 1)
        original_labels = a.eval(feed_dict={x: img, y_labels: la, keep_prob: 0.5})
        outlayer_labels = b.eval(feed_dict={x: img, y_labels: la, keep_prob: 0.5})
        if original_labels != outlayer_labels:
            la_dic[original_labels[0]][1] += 1
        else:
            la_dic[original_labels[0]][0] += 1
        if i%100 == 0:
            print("进行到第[%d]个" % (i))

    #召回率
    vaSum = currectNum = 0
    for key in la_dic.keys():
        va = la_dic[key]
        vaSum += va[0]+va[1]
        currectNum += va[0]
        if va[0]+va[1] != 0:
            recallRate = va[0] / (va[0]+va[1])
            print('[%s 召回率]：%f' % (labels_dict[key], recallRate))
    print('[正确率]: %f' % (currectNum/vaSum))
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from ./model/model-1500
进行到第[0]个
进行到第[100]个
进行到第[200]个
进行到第[300]个
进行到第[400]个
进行到第[500]个
进行到第[600]个
进行到第[700]个
进行到第[800]个
进行到第[900]个
进行到第[1000]个
进行到第[1100]个
[云 召回率]：0.466667
[京 召回率]：0.823529
[冀 召回率]：0.764706
[吉 召回率]：0.769231
[宁 召回率]：0.700000
[川 召回率]：0.979167
[晋 召回率]：0.894737
[桂 召回率]：0.984375
[沪 召回率]：0.862069
[津 召回率]：0.250000
[浙 召回率]：0.756757
[渝 召回率]：0.500000
[湘 召回率]：0.987342
[琼 召回率]：0.357143
[甘 召回率]：0.900000
[皖 召回率]：0.985294
[粤 召回率]：0.949495
[苏 召回率]：0.957143
[蒙 召回率]：1.000000
[豫 召回率]：0.984848
[贵 召回率]：1.000000
[赣 召回率]：0.955882
[辽 召回率]：0.150000
[鄂 召回率]：1.000000
[闽 召回率]：0.788462
[陕 召回率]：0.722222
[青 召回率]：1.000000
[鲁 召回率]：0.925000
[黑 召回率]：0.750000
[正确率]: 0.868043
